In [1]:
# Load Libs
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import nde_flows
import time
import csv
from pathlib import Path

print('numpy version:', np.__version__)
print('troch version:', torch.__version__)

numpy version: 1.16.2
troch version: 1.7.0+cu92


In [2]:
# Basic init
input_dim = 15 #nparams
context_dim = 400 # Nrb * len(detectors) * 2 # for RB

data_dir = 'data/'
model_dir = 'models/'

cuda = True 

In [4]:
# Model init
num_flow_steps = 15
base_transform_kwargs={
    'hidden_dim': 15,
    'num_transform_blocks': 10,
    'activation': 'elu',
    'dropout_probability': 0.0,
    'batch_norm': True,
    'num_bins': 8,
    'tail_bound': 1.0,
    'apply_unconditional_transform': False,
    'base_transform_type': 'rq-coupling'
} 

# Train init
lr = 0.0002
lr_annealing = True
anneal_method = lr_anneal_method = 'cosine'
total_epochs = epochs = 500
steplr_step_size = 80
steplr_gamma = 0.5
flow_lr = None

batch_size = 16
num_workers = 16
output_freq = 50
snr_annealing = False

This code uses the [nflows](https://github.com/bayesiains/nflows) package to implement the normalizing flow, which we take to be a [neural spline](https://arxiv.org/abs/2002.03712) flow. It makes use of the [PyTorch](https://pytorch.org) machine learning framework.

In [5]:
# Construct the neural network model.
model_creator = nde_flows.create_NDE_model

model = model_creator(input_dim=input_dim,
                      context_dim=context_dim,
                      num_flow_steps=num_flow_steps, 
                      base_transform_kwargs=base_transform_kwargs)

In [6]:
if cuda and torch.cuda.is_available():
    device = torch.device('cuda')
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    device = torch.device('cpu')
print('Device', device)

model.to(device)

print('\nInitial learning rate', lr)
print('Using cosine LR annealing.')
print('\nTraining for {} epochs'.format(epochs))

Device cuda

Initial learning rate 0.0002
Using cosine LR annealing.

Training for 500 epochs


In [7]:
# Set up the optimizer and scheduler.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=total_epochs,
)
print('\nModel hyperparameters:')
for key, value in model.model_hyperparams.items():
    if type(value) == dict:
        print(key)
        for k, v in value.items():
            print('\t', k, '\t', v)
    else:
        print(key, '\t', value)


Model hyperparameters:
input_dim 	 15
num_flow_steps 	 15
context_dim 	 400
base_transform_kwargs
	 hidden_dim 	 15
	 num_transform_blocks 	 10
	 activation 	 elu
	 dropout_probability 	 0.0
	 batch_norm 	 True
	 num_bins 	 8
	 tail_bound 	 1.0
	 apply_unconditional_transform 	 False
	 base_transform_type 	 rq-coupling


In [8]:
# Define a Wrapper.
# (demo dataset)
class WaveformDatasetTorch(Dataset):
    """Wrapper for a WaveformDataset to use with PyTorch DataLoader."""
    def __init__(self, data_dir, istrain):
        self.y = torch.from_numpy(np.load(data_dir+'dataset_{}_y.npy'.format('train' if istrain else 'test'))) # 使用numpy读取数据
        self.x = torch.from_numpy(np.load(data_dir+'dataset_{}_x.npy'.format('train' if istrain else 'test')))
        self.w = torch.from_numpy(np.load(data_dir+'dataset_{}_w.npy'.format('train' if istrain else 'test')))
        self.snr = torch.from_numpy(np.load(data_dir+'dataset_{}_snr.npy'.format('train' if istrain else 'test')))
        
        self.len = self.y.shape[0]
    
    def __getitem__(self, index):
        return (self.y[index], self.x[index], self.w[index], self.snr[index])

    def __len__(self):
        return self.len

In [9]:
# pytorch wrappers
wfd_train = WaveformDatasetTorch(data_dir, istrain=True)
wfd_test = WaveformDatasetTorch(data_dir, istrain=False)

In [10]:
# DataLoader objects
train_loader = DataLoader(
    wfd_train, batch_size=batch_size, shuffle=True, pin_memory=True,
    num_workers=num_workers,
    worker_init_fn=lambda _: np.random.seed(
        int(torch.initial_seed()) % (2**32-1)))
test_loader = DataLoader(
    wfd_test, batch_size=batch_size, shuffle=False, pin_memory=True,
    num_workers=num_workers,
    worker_init_fn=lambda _: np.random.seed(
        int(torch.initial_seed()) % (2**32-1)))

-  Train the network.

  - This trains a neural conditional density estimator with a neural spline coupling flow.
 

In [11]:
# Training
train_history = []
test_history = []
extrinsic_at_train = True
add_noise = False

print('Starting timer')
start_time = time.time()

epoch = 1
for epoch in range(epoch, epoch + epochs):
    print('Learning rate: {}'.format(
        optimizer.state_dict()['param_groups'][0]['lr']))    
    
    # Compute the loss
    train_loss = nde_flows.train_epoch(
        model,
        train_loader,
        optimizer,
        epoch,
        device,
        output_freq,
        add_noise,
        snr_annealing)
    test_loss = nde_flows.test_epoch(
        model,
        test_loader,
        epoch,
        device,
        add_noise,
        snr_annealing)
    

    if scheduler is not None:
        scheduler.step()

    epoch += 1
    train_history.append(train_loss)
    test_history.append(test_loss)

    # Log the history to file
    if model_dir is not None:
        p = Path(model_dir)
        p.mkdir(parents=True, exist_ok=True)

        # Make column headers if this is the first epoch
        if epoch == 1:
            with open(p / 'history.txt', 'w') as f:
                writer = csv.writer(f, delimiter='\t')
                writer.writerow([epoch, train_loss, test_loss])
        else:
            with open(p / 'history.txt', 'a') as f:
                writer = csv.writer(f, delimiter='\t')
                writer.writerow([epoch, train_loss, test_loss])
    break

print('Stopping timer.')
stop_time = time.time()
print('Training time (including validation): {} seconds'
      .format(stop_time - start_time))

Starting timer
Learning rate: 0.0002
Train Epoch: 1 [0/2000 (0%)]	Loss: 24.9883
Train Epoch: 1 [800/2000 (40%)]	Loss: 24.8968
Train Epoch: 1 [1600/2000 (80%)]	Loss: 23.2689
Train Epoch: 1 	Average Loss: 389.6537
Test set: Average loss: 386.1097

Stopping timer.
Training time (including validation): 107.74739027023315 seconds


- Save a model and optimizer to file.

In [12]:
# Save model
p = Path(model_dir)
p.mkdir(parents=True, exist_ok=True)

dict = {
    'model_hyperparams': model.model_hyperparams,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': epoch,
}

if scheduler is not None:
    dict['scheduler_state_dict'] = scheduler.state_dict()

torch.save(dict, p / 'model.pt')